In [1]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import time
import paho.mqtt.client as mqtt

In [2]:

# load the dataset
df = pd.read_csv('data/diabetes.csv')

In [3]:
# 1. Select the following 4 attributes (3 features + 1 class label) :
#   Glucose, BloodPressure, Insulin, Outcome
df = df[['Glucose', 'BloodPressure', 'Insulin', 'Outcome']]
print(df.head())

   Glucose  BloodPressure  Insulin  Outcome
0      148             72        0        1
1       85             66        0        0
2      183             64        0        1
3       89             66       94        0
4      137             40      168        1


In [4]:

# 2. Normalize Glucose, BloodPressure and Insulin to [0, 1] using MinMax.
scaler = MinMaxScaler()
df[['Glucose', 'BloodPressure', 'Insulin']] = scaler.fit_transform(df[['Glucose', 'BloodPressure', 'Insulin']])
print(df.head())

    Glucose  BloodPressure   Insulin  Outcome
0  0.743719       0.590164  0.000000        1
1  0.427136       0.540984  0.000000        0
2  0.919598       0.524590  0.000000        1
3  0.447236       0.540984  0.111111        0
4  0.688442       0.327869  0.198582        1


In [5]:

# 3. Store the new data (3 normalized features + 1 class label) in another dataset S.
df.to_csv('data/diabetes_normalized.csv', index=False)

In [ ]:
# 4. Modify the MQTT example to do the following:
#  • The publisher publishes records in S continuously. When it reaches the end of S, it continues to send from the beginning again.

# We create a client as the data publisher
mqttc = mqtt.Client()
# We are connecting to port 1883 and set *keepalive* parameter to 60; *keepalive* is the maximum waiting time in seconds.
mqttc.connect("mqtt.eclipseprojects.io", 1883, 60)

# We send the following readings repeatedly. Once we reach the end, we start from the beginning again.
data = df
period = len(data)

if __name__ == "__main__":
    print("Publishing...")
    
    index = 0
    while (index >= 0):
        # Get the current data reading to send out
        glucose = data['Glucose'][index % period]
        bloodPressure = data['BloodPressure'][index % period]
        insulin = data['Insulin'][index % period]
        outcome = data['Outcome'][index % period]
        
        # Publish the data reading
        mqttc.publish("diabetesData", str(glucose) + "," + str(bloodPressure) + "," + str(insulin) + "," + str(outcome))
        
        # We send the next reading after 2 seconds
        # time.sleep(0.5)
        index = index + 1